In [ ]:
import json
import os
import cv2
import shutil
import numpy as np

path_face = os.getcwd() + "/"
os.getcwd()

In [ ]:
# test.txt
try:
    os.remove('./cfg/test_img.txt')
except:
    pass
path_test = path_face + "test_img/"
files = os.listdir(path_test)
files.sort()
for file in files:
    with open('./cfg/test_img.txt', 'a', encoding = 'utf-8') as f:
        f.write(path_test + file + "\n")

In [ ]:
os.chdir("../")
os.getcwd()

In [ ]:
model = "181000"
thresh = 0.1
order = "./darknet detector test Bone_Scan_1207/cfg/1207.data Bone_Scan_1207/cfg/yolov4-tiny-obj.cfg Bone_Scan_1207/cfg/weights/yolov4-tiny-obj_" + str(model) + ".weights -thresh " + str(thresh) + " -ext_output -dont_show -out Bone_Scan_1207/result.json < Bone_Scan_1207/cfg/test_img.txt > Bone_Scan_1207/result.txt"
os.system(order)

savepath = "./Bone_Scan_1207/result_" + model + "/"
try:
    shutil.rmtree(savepath)
except:
    pass
os.mkdir(savepath)

path = "./Bone_Scan_1207/test_img/"
txtsavepath = "./Bone_Scan_1207/test_label/"
try:
    shutil.rmtree(txtsavepath)
except:
    pass
os.mkdir(txtsavepath)
images = os.listdir(path)
images.sort()
print(len(images))
with open("./Bone_Scan_1207/result.json", "r") as f:
    json_data = json.load(f)
print(len(json_data))

In [ ]:
for i in range(len(json_data)):
    dict_data = json_data[i]
    filename = dict_data["filename"]
    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    filename = filename[filename.rfind("/") + 1:]
    print(filename)
    objects = dict_data['objects']
    if objects == []:
        with open(txtsavepath + filename.replace("png", "txt"), 'a', encoding = 'utf-8') as f:
            f.write("\n")
    for o in objects:
        class_id = o["class_id"]
#         print("class:", class_id)
        coordinates = o['relative_coordinates']
        x_c = round(float(coordinates['center_x']) * 292)
        y_c = round(float(coordinates['center_y']) * 346)
        w = round(float(coordinates['width']) * 292)
        h = round(float(coordinates['height']) * 346)
        x = round(x_c - w / 2)
        y = round(y_c - h / 2)
        row = "{} {} {} {} {} {}".format(class_id,
                                         o['confidence'],
                                         coordinates['center_x'],
                                         coordinates['center_y'],
                                         coordinates['width'],
                                         coordinates['height'])
        print(row)
        with open(txtsavepath + filename.replace("png", "txt"), 'a', encoding = 'utf-8') as f:
            f.write(row + "\n")
        if class_id == 0:
            img = cv2.rectangle(img, (x, y),
                                (x + w, y + h), (255, 0, 0), 2)
        elif class_id == 1:
            img = cv2.rectangle(img, (x, y),
                                (x + w, y + h), (255, 255, 0), 2)
        elif class_id == 2:
            img = cv2.rectangle(img, (x, y),
                                (x + w, y + h), (0, 255, 0), 2)
    cv2.imwrite(savepath + filename, img[:,:,::-1])
    print("--------------------------------------------------")
    print()

In [ ]:
os.chdir("./Bone_Scan_1207/")
os.getcwd()

In [ ]:
path_images = "./test_img/"
path_labels = "./test_label/"
savepath = "./final_" + model + "/"
try:
    shutil.rmtree(savepath)
except:
    pass
os.mkdir(savepath)

images = os.listdir(path_images)
images.sort()
labels = os.listdir(path_labels)
labels.sort()

def mat_inter(box1, box2):
    x01 = box1[0]
    y01 = box1[1]
    x02 = box1[2]
    y02 = box1[3]
    x11 = box2[0]
    y11 = box2[1]
    x12 = box2[2]
    y12 = box2[3]

    lx = abs((x01 + x02) / 2 - (x11 + x12) / 2)
    ly = abs((y01 + y02) / 2 - (y11 + y12) / 2)
    sax = abs(x01 - x02)
    sbx = abs(x11 - x12)
    say = abs(y01 - y02)
    sby = abs(y11 - y12)
    if lx <= (sax + sbx) / 2 and ly <= (say + sby) / 2:
        return True
    else:
        return False
    
def solve_coincide(box1, box2):
    if mat_inter(box1, box2) == True:
        x01 = box1[0]
        y01 = box1[1]
        x02 = box1[2]
        y02 = box1[3]
        x11 = box2[0]
        y11 = box2[1]
        x12 = box2[2]
        y12 = box2[3]
        col = min(x02, x12) - max(x01, x11)
        row = min(y02, y12) - max(y01, y11)
        intersection = col * row
        area1 = (x02 - x01) * (y02 - y01)
        area2 = (x12 - x11) * (y12 - y11)
#         coincide = intersection / (area1 + area2 - intersection)
        coincide1 = intersection / (area1)
        coincide2 = intersection / (area2)
        coincide = max(coincide1, coincide2)
        return coincide
    else:
        return False
    
def drawline(img, pt1, pt2, color, thickness = 1, style = 'dotted', gap = 20):
    dist = ((pt1[0] - pt2[0]) ** 2 + (pt1[1] - pt2[1]) ** 2) ** 0.5
    pts = []
    for i in  np.arange(0, dist, gap):
        r = i/dist
        x = int((pt1[0] * (1 - r) + pt2[0] * r) + .5)
        y = int((pt1[1] * (1 - r) + pt2[1] * r) + .5)
        p  =  (x, y)
        pts.append(p)

    if style == 'dotted':
        for p in pts:
            cv2.circle(img, p, thickness, color,  - 1)
    else:
        s = pts[0]
        e = pts[0]
        i = 0
        for p in pts:
            s = e
            e = p
            if i % 2 == 1:
                cv2.line(img, s, e, color, thickness)
            i += 1
            
def drawpoly(img, pts, color, thickness = 1, style = 'dotted'):
    s = pts[0]
    e = pts[0]
    pts.append(pts.pop(0))
    for p in pts:
        s = e
        e = p
        drawline(img, s, e, color, thickness, style)
        
def drawrect(img, pt1, pt2, color, thickness = 1, style = 'dotted'):
    pts = [pt1, (pt2[0], pt1[1]), pt2, (pt1[0], pt2[1])] 
    drawpoly(img, pts, color, thickness, style)
    
def draw_line(img, class_id, x1, y1, x2, y2):
    if class_id == 0:
        img = cv2.rectangle(img, (x1, y1),
                            (x2, y2), (255, 0, 0), 2)
    elif class_id == 1:
        img = cv2.rectangle(img, (x1, y1),
                            (x2, y2), (255, 255, 0), 2)
    elif class_id == 2:
        img = cv2.rectangle(img, (x1, y1),
                            (x2, y2), (0, 255, 0), 2)
        
def draw_dottedline(img, class_id, x1, y1, x2, y2):
    if class_id == 0:
        img = drawrect(img, (x1, y1),
                       (x2, y2), (255, 0, 0), 2, 'dotted')
    elif class_id == 1:
        img = drawrect(img, (x1, y1),
                        (x2, y2), (255, 255, 0), 2, 'dotted')
    elif class_id == 2:
        img = drawrect(img, (x1, y1),
                        (x2, y2), (0, 255, 0), 2, 'dotted')        

In [ ]:
for a in range(len(images)):
    img = cv2.imread(path_images + images[a])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    with open(path_labels + labels[a], "r", encoding = 'utf-8') as f:
        lines = f.readlines()
    if lines == ['\n']:
        img = cv2.imread(path_images + images[a])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        cv2.imwrite(savepath + images[a], img[:,:,::-1])
        continue
        
    boxs = []
    for line in lines:
        box = line.replace("\n", "").split(" ")
        x_c = round(float(box[2]) * 292)
        y_c = round(float(box[3]) * 346)
        w = round(float(box[4]) * 292)
        h = round(float(box[5]) * 346)
        x1 = round(x_c - w / 2)
        y1 = round(y_c - h / 2)
        x2 = x1 + w
        y2 = y1 + h
        box = [int(box[0]), float(box[1]), x1, y1, x2, y2]
        boxs.append(box)

    # 獲得有重合的部份
    index_coincide = [] # 二維list
    for i in range(len(boxs)):
        for j in range(i + 1, len(boxs)): # 比過的下次略過，避免重複繪製
            coincide = solve_coincide(boxs[i][2:], boxs[j][2:])
            if coincide > 0.5:
                index_coincide.append([i, j])

    # 繪製無重合部份
    list_coincide = [i for item in index_coincide for i in item] # 有重合的一維list
    for i in range(len(boxs)):
        if i not in list_coincide: # 不在list內繪製實線
            draw_line(img, boxs[i][0], boxs[i][2], boxs[i][3], boxs[i][4], boxs[i][5])

    # 繪製重合部份
    for idx in index_coincide: # confidence較大的繪製實線，較小則繪製虛線
        box1 = boxs[idx[0]]
        box2 = boxs[idx[1]]
        if box1[1] > box2[1]:
            draw_line(img, box1[0], box1[2], box1[3], box1[4], box1[5])
            draw_dottedline(img, box2[0], box2[2], box2[3], box2[4], box2[5])
        else:
            draw_line(img, box2[0], box2[2], box2[3], box2[4], box2[5])
            draw_dottedline(img, box1[0], box1[2], box1[3], box1[4], box1[5])
    cv2.imwrite(savepath + images[a], img[:,:,::-1])